In [1]:
#pip install nsepy

In [8]:
import time
import datetime
from datetime import date

import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dateutil.relativedelta import relativedelta



from nsepy import get_history
from nsepy.derivatives import get_expiry_date

In [9]:
# lets try to download for a single option first; manually specify dates
sample_opt1 = get_history(symbol = "NIFTY",
                        start = date(2004,11,1), end = date(2005,1,27),
                        index = True,
                        option_type = 'PE',
                        strike_price = 2000,
                        expiry_date = date(2005,1,27))

print(sample_opt1.shape)
sample_opt1.head(n=3)

(38, 16)


,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2004-12-03,NIFTY,2005-01-27,PE,2000.0,61.95,65.00,57.5,65.00,64.90,65.00,312,128725000.0,NaN,46800,46800,1996.20
2004-12-06,NIFTY,2005-01-27,PE,2000.0,52.10,62.30,52.1,55.40,55.40,55.40,170,70036000.0,NaN,75400,28600,1993.15
2004-12-07,NIFTY,2005-01-27,PE,2000.0,59.00,62.85,59.0,62.85,62.85,62.85,3,1237000.0,NaN,75200,-200,1992.70


In [20]:
current_date = date(2007,1,1)
print(current_date)
print(type(current_date))

2007-01-01
<class 'datetime.date'>


In [21]:
start_date = current_date + relativedelta(months = -2)
start_month = current_date.month

print(start_date)
print(type(start_date))
print(start_month)
start_yr = start_date.year
print(start_yr)


2006-11-01
<class 'datetime.date'>
1
2006


In [22]:
end_month = current_date.month
end_yr =current_date.year

print(end_month)
print(end_yr)

expiry_date = max(get_expiry_date(year=end_yr,month=end_month))
print('Expiry_date:',expiry_date,type(expiry_date))


1
2007
Expiry_date: 2007-01-25 <class 'datetime.date'>


In [31]:
sample_opt2 = get_history(symbol = "NIFTY",
                        start = start_date, end = expiry_date,
                        index = True,
                        option_type = "PE",
                        strike_price = 2000,
                        expiry_date = expiry_date)
print(sample_opt2.shape)

print(start_date,expiry_date)

sample_opt2.head(n=3)

(0, 16)
2006-11-01 2007-01-25


,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,


In [25]:
# define and month year range to loop over
month_list = np.arange(1, 13, step = 1); print(month_list)
yr_list = np.arange(2018, 2023, step = 1 ); print(yr_list)

[ 1  2  3  4  5  6  7  8  9 10 11 12]
[2018 2019 2020 2021 2022]


In [26]:
print(type(month_list))
print(type(yr_list))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [27]:
# empty dataframe
nifty_data = pd.DataFrame() # to use in the loop
option_data = pd.DataFrame() # to store output

In [28]:
nifty_data

""


In [29]:
option_data

""


In [32]:
# break the loop into 2 parts to avoid having to rerun everything if there are querying errors
for yr in yr_list:
    # loop through all the months and years
    counter = 0
    print('Year: ', yr)
    for mnth in month_list:
        current_dt = date(yr, mnth, 1)
        start_dt = current_dt + relativedelta(months = -2)
        end_dt = max(get_expiry_date(year = yr, month = mnth))
        
        # print('current: ', current_dt)
        # print('start: ', start_dt)
        # print('end: ', end_dt)
        
        # get nifty futures data
        nifty_fut = get_history(symbol = 'NIFTY',
                               start = start_dt, end = end_dt,
                               index = True,
                               expiry_date = end_dt)
        nifty_data = nifty_data.append(nifty_fut)
        
        # calculate high and low values for each month; round off to get strike prices
        high = nifty_fut['Close'].max()
        high = int(round(high/100)*100) + 1000# ; print('High:', high)
        
        low = nifty_fut['Close'].min()
        low = int(round(low/100)*100) + 1000# ; print('Low :', low)
        
        for strike in range(low, high, 100): # start, stop, step
            """
            get daily closing nifty index option prices for 3 months 
            over the entire range 
            """
            # time.sleep(random.randint(10,20)) # pause for random interval so as to not overwhelm the site
            nifty_opt = get_history(symbol = 'NIFTY',
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'PE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
            #time.sleep(random.randint(20,50)) # pause for random interval so as to not overwhelm the site
            nifty_opt = get_history(symbol = 'NIFTY',
                                   start = start_dt, end = end_dt,
                                   index = True, option_type = 'CE',
                                   strike_price = strike,
                                   expiry_date = end_dt)
            
            option_data = option_data.append(nifty_opt)
            
        counter+=1
        print('Months: ', counter)
        # print(month)       

Year:  2018


C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nifty_data = nifty_data.append(nifty_fut)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\

Months:  1


C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nifty_data = nifty_data.append(nifty_fut)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\

Months:  2


...

Months:  11


C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nifty_data = nifty_data.append(nifty_fut)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\

C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)
C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)


Months:  12


C:\Users\HP\AppData\Local\Temp\ipykernel_13768\3400827828.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  option_data = option_data.append(nifty_opt)


In [33]:
print(nifty_data.shape)
nifty_data.tail(n=3)

(3631, 6)


,Open,High,Low,Close,Volume,Turnover
Date,,,,,,
2022-12-27,18089.80,18149.25,17967.45,18132.3,214257094,1.300064e+11
2022-12-28,18084.75,18173.10,18068.35,18122.5,193873167,1.399759e+11
2022-12-29,18045.70,18229.70,17992.80,18191.0,281052828,1.846605e+11


In [34]:
print(option_data.shape)
option_data.tail(n=3)

(96612, 16)


,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2022-12-27,NIFTY,2022-12-29,CE,19700.0,0.8,0.85,0.45,0.5,0.45,0.5,12964,12769934000.0,394000.0,438400,22000,18132.3
2022-12-28,NIFTY,2022-12-29,CE,19700.0,0.5,0.65,0.25,0.35,0.25,0.35,5483,5400865000.0,110000.0,425750,-12650,18122.5
2022-12-29,NIFTY,2022-12-29,CE,19700.0,0.4,0.5,0.05,0.05,0.05,18191.0,11715,11539466000.0,191000.0,275050,-150700,18191.0


In [35]:
option_data.to_csv("nifty_5 yr_data.csv")

In [36]:
nifty_data.to_csv("nifty_5 yr_option_data.csv")